In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


Tải thư viện và import

In [ ]:
# !pip -q install torchmetrics==1.4.0
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchmetrics.regression import MeanAbsoluteError
from pathlib import Path

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 23.4 MB/s eta 0:00:00


- ROOT: thư mục gốc của dự án trên Drive.

- DATA_NPZ: file dữ liệu dạng .npz (nhiều mảng NumPy nén) sẽ được load.

- CKPT_DIR: thư mục để lưu checkpoint (trọng số mạng).

- CKPT_PATH: file cụ thể để lưu “model tốt nhất” (best checkpoint).



- BATCH_SIZE: kích thước batch (số chuỗi để huấn luyện) khi huấn luyện/đánh giá.

- EPOCHS: số vòng (epoch) tối đa để huấn luyện.

- PATIENCE: số epoch cho phép “không cải thiện” trước khi dừng sớm (early stopping).

In [ ]:
# ========== CẤU HÌNH ==========
ROOT = Path("/content/drive/MyDrive/hurricane-trajectory")
DATA_NPZ = ROOT / "data/processed_data.npz"
CKPT_DIR = ROOT / "models"
CKPT_PATH = CKPT_DIR / "best_lstm_manual.pt"

BATCH_SIZE = 64
EPOCHS = 40
PATIENCE = 10  # early stopping

Class StormSeqDataset thừa kế Dataset:
+ Hàm nhận đầu vào ép kiểu và kiểm tra shape trước queeze chuyển dữ liệu thành tensor kiểu float32, chỉnh shape của dataset train (bỏ chiều thứ 2 nếu kích thước là 1).
+ Hàm trả về số phần tử trong mẫu (số dòng).
+ Hàm truy cập gọi từng mẫu thông qua i cho DataLoader tạo tensor batch để train.

In [ ]:
class StormSeqDataset(Dataset):
    def __init__(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.X = torch.tensor(X, dtype=torch.float32)
        if y.ndim >= 3 and y.shape[1] == 1:
            y = np.squeeze(y, axis=1)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]



    LSTM công thức:
      + Forget gate: f_t = σ(U_f x_t + W_f h_{t-1} + b_f)

      + Input gate:i_t = σ(U_i x_t + W_i h_{t-1} + b_i)

      + Output gate:o_t = σ(U_o x_t + W_o h_{t-1} + b_o)

      + Internal sate: g_t = tanh(U_g x_t + W_g h_{t-1} + b_g)

      c_t = f_t ⊙ c_{t-1} + i_t ⊙ g_t

      h_t = o_t ⊙ tanh(c_t)
  - Hàm khởi tạo các tham số cho công thức và xây dựng các công thức tính cho mô hình LSTM: f_t, i_t, o_t, g_t,...

In [ ]:
class _ManualLSTMCell(nn.Module):
    def __init__(self, in_dim: int, hidden: int):
        super().__init__()
        self.hidden = hidden
        # U*: input->hidden (có bias)
        self.Uf = nn.Linear(in_dim, hidden, bias=True)
        self.Ui = nn.Linear(in_dim, hidden, bias=True)
        self.Uo = nn.Linear(in_dim, hidden, bias=True)
        self.Ug = nn.Linear(in_dim, hidden, bias=True)
        # W*: hidden->hidden (không bias)
        self.Wf = nn.Linear(hidden, hidden, bias=False)
        self.Wi = nn.Linear(hidden, hidden, bias=False)
        self.Wo = nn.Linear(hidden, hidden, bias=False)
        self.Wg = nn.Linear(hidden, hidden, bias=False)

        # Khởi tạo
        for lin in [self.Uf, self.Ui, self.Uo, self.Ug]:
            nn.init.xavier_uniform_(lin.weight)
            nn.init.zeros_(lin.bias)
        for lin in [self.Wf, self.Wi, self.Wo, self.Wg]:
            nn.init.orthogonal_(lin.weight)
        # Forget bias dương để khuyến khích "nhớ" lúc đầu
        with torch.no_grad():
            self.Uf.bias.fill_(1.0)

    def forward(self, x_t, h_prev, c_prev):
        f_t = torch.sigmoid(self.Uf(x_t) + self.Wf(h_prev))
        i_t = torch.sigmoid(self.Ui(x_t) + self.Wi(h_prev))
        o_t = torch.sigmoid(self.Uo(x_t) + self.Wo(h_prev))
        g_t = torch.tanh(   self.Ug(x_t) + self.Wg(h_prev))
        c_t = f_t * c_prev + i_t * g_t
        h_t = o_t * torch.tanh(c_t)
        return h_t, c_t


1. Hàm constructor khởi tạo lstm và tính output đầu ra với tham số:
-in_dim: số đặc trưng ở mỗi bước thời gian đầu vào (số lượng features).

-hidden: kích thước tầng ẩn của LSTM (số neural ở mỗi layer).

-num_layers: số lớp LSTM chồng lên nhau.

-out_dim: số chiều đầu ra (ví dụ 2 toạ độ- kinh độ vĩ độ).

-dropout:(một phần kết nối/nút bị tắt ngẫu nhiên) regularization - tỉ lệ dropout giữa các lớp LSTM để chống overfitting.

- Tạo num_layers cell; layer 0 nhận in_dim, các layer sau nhận hidden.


- Truyền inp qua từng layer; với layer không cuối, áp dropout lên h_l trước khi đưa xuống layer tiếp theo.

- Sau khi đi hết thời gian, lấy h của layer cuối ở timestep cuối → qua Linear head để ra vector dự đoán [B, out_dim].

2. Hàm forward:
- Input vào forward: batch chuỗi x có shape [B, T, F] (F = in_dim).
- Lấy thiết bị mà x đang nằm trên: CPU hay GPU
- Lấy kiểu dữ liệu của x

- Mỗi layer trong self.cells là 1 LSTM tự code:

-Mỗi epoch forward: với từng t từ 0...T−1:

-Tại mỗi t, truyền x/tín hiệu qua từng layer; giữa các layer (trừ layer cuối) áp self.do_mid (dropout).

- Sau khi đi hết T bước, lấy hidden của layer cuối ở timestep cuối (shape [B, hidden]), đưa qua self.head để ra [B, out_dim].

In [ ]:
class LSTMFromScratchForecaster(nn.Module):
    """
    Stacked LSTM từ _ManualLSTMCell. Lấy h ở timestep cuối -> Linear head ra out_dim.
    """
    def __init__(self, in_dim, hidden=20, num_layers=2, out_dim=2, dropout=0.2):
        super().__init__()
        self.hidden = hidden
        self.num_layers = num_layers
        self.dropout_p = dropout if num_layers > 1 else 0.0

        self.cells = nn.ModuleList([
            _ManualLSTMCell(in_dim if l == 0 else hidden, hidden)
            for l in range(num_layers)
        ])
        self.dropout = nn.Dropout(self.dropout_p) if self.dropout_p > 0 else nn.Identity()
        self.head = nn.Linear(hidden, out_dim)

        nn.init.xavier_uniform_(self.head.weight)
        nn.init.zeros_(self.head.bias)

    def forward(self, x):
        # x: [B, T, F]
        B, T, _ = x.size()
        device = x.device
        dtype = x.dtype

        hs = [torch.zeros(B, self.hidden, device=device, dtype=dtype) for _ in range(self.num_layers)]
        cs = [torch.zeros(B, self.hidden, device=device, dtype=dtype) for _ in range(self.num_layers)]

        for t in range(T):
            inp = x[:, t, :]
            for l, cell in enumerate(self.cells):
                h_l, c_l = cell(inp, hs[l], cs[l])
                hs[l], cs[l] = h_l, c_l
                # dropout giữa các layer (không áp cho layer cuối)
                inp = self.dropout(h_l) if (l < self.num_layers - 1) else h_l

        last_h = hs[-1]          # [B, H] tại timestep cuối
        return self.head(last_h) # [B, out_dim]


Hàm train và evaluation:

loader: DataLoader cung cấp các batch (xb, yb).

model: mô hình cần chạy.

crit: hàm loss (nn.MSELoss()).

opt: optimizer (chỉ dùng khi train).

device: CPU/GPU để đưa tensor lên.

train_mode: True để huấn luyện, False để đánh giá.

mae_metric: bộ đo min absolute erro (từ torchmetrics).
- Set train_mode hoặc evaluation (bỏ dropout), autograd(đạo hàm).
- Khởi tạo total_loss và n_sample để tính average epoch loss chuẩn xác hơn
- Đưa dữ liệu xb,yb để dự đoán, tính loss(MSE); xoá bỏ gradient cũ nếu train, tính gradient mới .backpropagation, giới hạn độ lớn của gradient, update, cập nhật MAE cho batch hiện tại.


In [ ]:
@torch.no_grad()
def _eval_epoch(loader, model, crit, device, mae_metric):
    model.eval()
    total_loss, n_samples = 0.0, 0
    mae_metric.reset()
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = crit(pred, yb)
        mae_metric.update(pred, yb)
        total_loss += loss.item() * xb.size(0)
        n_samples += xb.size(0)
    return total_loss / max(n_samples, 1), mae_metric.compute()

def _train_epoch(loader, model, crit, opt, device, mae_metric):
    model.train()
    total_loss, n_samples = 0.0, 0
    mae_metric.reset()
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = crit(pred, yb)

        opt.zero_grad(set_to_none=True)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()

        mae_metric.update(pred, yb)
        total_loss += loss.item() * xb.size(0)
        n_samples += xb.size(0)
    return total_loss / max(n_samples, 1), mae_metric.compute()

def run_epoch(loader, model, crit, opt, device, train_mode, mae_metric):
    if train_mode:
        return _train_epoch(loader, model, crit, opt, device, mae_metric)
    else:
        return _eval_epoch(loader, model, crit, device, mae_metric)


Hàm main:
1. Load dữ liệu từ .npz; tách tập train, validation, test; lấy danh sách đặc trưng đầu vào(fearture).

2.Xác định chiều ra của mô hình

3.Tạo DataLoader để cho train, validation, test dùng hàm StormSeqDataset() ở trên cùng BATCH_SIZE; xáo trộn dữ liệu mỗi epoch khi train.
4. Thiết bị sử dụng

5. Khởi tạo mô hình, tính các thông số:
- Tính MSE.
- Optimize với learning rate(LR) nếu cần, giảm đi một nửa LR nếu sau 3 epoch liên tiếp không giảm va_loss.
- Tính MAE.
6. Tạo thư mục checkpoint lưu best loss xem khi nào model chạy tốt nhất và số epoch không cải thiện.
7. Lưu “model tốt nhất” & early stopping:
- Nếu va_loss nhỏ hơn best_loss, cập nhật best_loss, reset bad_epochs
- Lưu checkpoint: trọng số mô hình, meta về cột/đặc trưng
- Nếu va_loss lớn hơn best_loss thì tăng bad_epochs đến PATIENCE thì dừng
8. Đánh giá trên tập Test
- Load best model
- Nạp trọng số tốt nhất  vào model
chạy trên tập Test để lấy MSE, MAE.

In [ ]:
def main():
    if not DATA_NPZ.exists():
        raise FileNotFoundError(f"Không tìm thấy file dữ liệu: {DATA_NPZ}")

    # 1) Load dữ liệu
    npz = np.load(DATA_NPZ, allow_pickle=True)
    X_train, y_train = npz["X_train"], npz["y_train"]
    X_valid, y_valid = npz["X_valid"], npz["y_valid"]
    X_test,  y_test  = npz["X_test"],  npz["y_test"]
    INPUT_FEATURES = list(npz["INPUT_FEATURES"]) if "INPUT_FEATURES" in npz.files else None
    TARGET_FEATURES = list(npz["TARGET_FEATURES"]) if "TARGET_FEATURES" in npz.files else None

    # 2) out_dim
    out_dim = len(TARGET_FEATURES) if TARGET_FEATURES else (y_train.shape[-1] if y_train.ndim >= 2 else 1)
    in_dim = X_train.shape[2]

    # 3) DataLoader
    train_loader = DataLoader(StormSeqDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(StormSeqDataset(X_valid, y_valid), batch_size=BATCH_SIZE, shuffle=False)
    test_loader  = DataLoader(StormSeqDataset(X_test,  y_test),  batch_size=BATCH_SIZE, shuffle=False)

    # 4) Thiết bị
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Sử dụng thiết bị: {device}")

    # 5) Model + tối ưu + metric
    model = LSTMFromScratchForecaster(in_dim=in_dim, hidden=20, num_layers=2, out_dim=out_dim, dropout=0.2).to(device)
    crit = nn.MSELoss()
    opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)  # <= inline
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=3)  # <= inline
    mae_metric = MeanAbsoluteError().to(device)

    # 6) Checkpoint dir
    CKPT_DIR.mkdir(parents=True, exist_ok=True)

    # 7) Train + Early stopping
    best_loss, bad_epochs = float("inf"), 0
    print("\nBắt đầu huấn luyện...")
    for ep in range(1, EPOCHS + 1):
        tr_loss, _ = run_epoch(train_loader, model, crit, opt, device, True, mae_metric)
        va_loss, va_mae = run_epoch(valid_loader, model, crit, None, device, False, mae_metric)

        sched.step(va_loss)

        print(f"Epoch {ep:02d} | Train Loss {tr_loss:.6f} | Valid Loss {va_loss:.6f} | Valid MAE {va_mae:.6f}")

        if va_loss < best_loss - 1e-12:
            best_loss, bad_epochs = va_loss, 0
            torch.save(
                {
                    "model_state_dict": model.state_dict(),
                    "input_features": INPUT_FEATURES,
                    "target_features": TARGET_FEATURES,
                    "in_dim": in_dim,
                    "out_dim": out_dim,
                    "config": {"hidden": 20, "num_layers": 2, "dropout": 0.2},
                },
                CKPT_PATH,
            )
            print(" -> Đã lưu model tốt nhất.")
        else:
            bad_epochs += 1
            if bad_epochs >= PATIENCE:
                print("Early stopping do không cải thiện trên tập validation.")
                break

    # 8) Test
    print("\nĐánh giá trên tập test...")
    checkpoint = torch.load(CKPT_PATH, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    test_loss, test_mae = run_epoch(test_loader, model, crit, None, device, False, mae_metric)
    print(f"[KẾT QUẢ TEST] MSE: {test_loss:.6f} | MAE: {test_mae:.6f}")

# chạy
main()


Sử dụng thiết bị: cuda

Bắt đầu huấn luyện...
Epoch 01 | Train Loss 0.014340 | Valid Loss 0.008068 | Valid MAE 0.060328
 -> Đã lưu model tốt nhất.
Epoch 02 | Train Loss 0.005265 | Valid Loss 0.003355 | Valid MAE 0.041340
 -> Đã lưu model tốt nhất.
Epoch 03 | Train Loss 0.002959 | Valid Loss 0.002648 | Valid MAE 0.035959
 -> Đã lưu model tốt nhất.
Epoch 04 | Train Loss 0.002507 | Valid Loss 0.002293 | Valid MAE 0.033579
 -> Đã lưu model tốt nhất.
Epoch 05 | Train Loss 0.002261 | Valid Loss 0.002136 | Valid MAE 0.032391
 -> Đã lưu model tốt nhất.
Epoch 06 | Train Loss 0.002053 | Valid Loss 0.001887 | Valid MAE 0.030350
 -> Đã lưu model tốt nhất.
Epoch 07 | Train Loss 0.001892 | Valid Loss 0.001714 | Valid MAE 0.028881
 -> Đã lưu model tốt nhất.
Epoch 08 | Train Loss 0.001718 | Valid Loss 0.001581 | Valid MAE 0.027805
 -> Đã lưu model tốt nhất.
Epoch 09 | Train Loss 0.001587 | Valid Loss 0.001567 | Valid MAE 0.028602
 -> Đã lưu model tốt nhất.
Epoch 10 | Train Loss 0.001502 | Valid Loss 0